<a href="https://colab.research.google.com/github/UtG1209/Supply-Chain-Optimization/blob/main/Stocks%20procurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 36.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pulp import *
import numpy as np
from scipy.optimize import minimize
import math
import time
from datetime import datetime

In [ ]:
# Demand
df_demand = pd.read_csv('df_demandsku.csv', index_col=0)
print("{:,} total demand".format(df_demand.DEMAND.sum()))
df_demand.head()

9,170 total demand


,SKU,DEMAND
0,D1,218
1,D2,277
2,D3,62
3,D4,142
4,D5,146


In [ ]:
df_costsku = pd.read_csv('df_costsku.csv', index_col=0)
print("{:,} average cost per carton".format(df_costsku.COST.mean()))
df_costsku.head()

186.1 average cost per carton


,SKU,COST
0,D1,181
1,D2,126
2,D3,144
3,D4,238
4,D5,315


In [ ]:
A = -0.3975
b = 42.250

In [ ]:
def objective(R):
    result = 0
    for i in range(60):
        # TR Costs
        result += (A*(df_demand.loc[i,'DEMAND']/R[i]) + b) * R[i]
        # Capital Costs
        result += (df_demand.loc[i,'DEMAND']/(2*R[i])) * df_costsku.loc[i,'COST']*0.125
        # Storage Costs
        result += (df_demand.loc[i,'DEMAND']/(2*R[i])) * 12 * 480/2000
    return result

In [ ]:
# Initialize constraints list
cons = []
# Maximum Inventory
def constraint1(R):
    loop = 0
    for i in range(60):
        loop += R[i]
    result = 480 - loop
    return result
cons.append({'type':'ineq','fun':constraint1})

In [ ]:
# Add Order Size Constraints
for i in range(60):
    # Minimum Order Quantity
    c2 = lambda R : (df_demand.loc[i,'DEMAND']/R[i]) - 1
    cons.append({'type':'ineq','fun':c2})
    # Maximum Order Quantity
    c3 = lambda R : 400 - (df_demand.loc[i,'DEMAND']/R[i])
    cons.append({'type':'ineq','fun':c3})

In [ ]:
# All SKU replenished 1 time
R0 = [2 for i in range(60)]
print("${:,} total cost for initial guessing".format(objective(R0).round(1)))

$63,206.7 total cost for initial guessing


In [ ]:
# Bound vector
b_vector = (1, 365)
bnds = tuple([b_vector for i in range(60)])

In [ ]:
start = time.time()
sol = minimize(objective, R0, method = 'SLSQP', bounds=bnds, constraints = cons, options={'maxiter': 100})
exec_time = (time.time()-start)
print("Execution time is {}s for 100 iterations".format(exec_time))

Execution time is 41.50133681297302s for 100 iterations


In [ ]:
# Initial solution
sol_init = sol.x
# Take the floor of the solution to have an integer as number of replenishment and never exceed stock limit
sol_final = [math.floor(i) for i in sol_init]

In [ ]:
print(('''For {} Iterations
-> Initial Solution: ${:,}
-> Integer Solution: ${:,}
''').format(100, sol.fun.round(1), objective(sol_final).round(1)))

For 100 Iterations
-> Initial Solution: $28,991.9
-> Integer Solution: $29,221.3



In [ ]:

print("Maximum inventory level with continuous number of replenishment: {}".format(sum(sol_init)))
print("Maximum inventory level with continuous number of replenishment: {}".format(sum(sol_final)))

Maximum inventory level with continuous number of replenishment: 386.2361790436466
Maximum inventory level with continuous number of replenishment: 356


In [ ]:
start = time.time()
sol = minimize(objective, R0, method = 'SLSQP', bounds=bnds, constraints = cons, options={'maxiter': 500})
exec_time = (time.time()-start)
print("Execution time is {}s for 500 iterations".format(exec_time))

Execution time is 36.39472317695618s for 500 iterations


In [ ]:
# Initial solution
sol_init2 = sol.x
# Take the ceiling of the solution to have an integer as number of replenishment
sol_final2 = [math.ceil(i) for i in sol_init2]

In [ ]:
print(('''For {} Iterations
-> Initial Solution: ${:,}
-> Integer Solution: ${:,}
''').format(100, sol.fun.round(1), objective(sol_final2).round(1)))

For 100 Iterations
-> Initial Solution: $28,991.9
-> Integer Solution: $29,126.5



In [ ]:
import pandas as pd
from scipy.optimize import minimize

# Load data
df_costsku = pd.read_csv('df_costsku.csv')
df_demand = pd.read_csv('df_demandsku.csv')

# Constants
TRANSPORT_COST_COEFF = -0.3975
TRANSPORT_COST_INTERCEPT = 42.250
COST_OF_CAPITAL = 0.125
STORAGE_COST_PER_CARTON = 12 * 480 / 2000  # Monthly storage cost

# Objective function to minimize total cost
def objective(replenishments):
    total_cost = 0
    for i in range(len(df_demand)):
        annual_demand = df_demand.loc[i, 'DEMAND']
        replenishment_qty = annual_demand / replenishments[i]

        # Transportation cost
        transport_cost = (TRANSPORT_COST_COEFF * replenishment_qty + TRANSPORT_COST_INTERCEPT) * replenishments[i]
        # Capital cost
        capital_cost = (annual_demand / (2 * replenishments[i])) * df_costsku.loc[i, 'COST'] * COST_OF_CAPITAL
        # Storage cost
        storage_cost = (annual_demand / (2 * replenishments[i])) * STORAGE_COST_PER_CARTON

        total_cost += transport_cost + capital_cost + storage_cost
    return total_cost

# Constraints
def inventory_constraint(replenishments):
    return 480 - sum(replenishments)  # Ensure total inventory doesn't exceed capacity

def min_order_qty_constraint(replenishments):
    return [df_demand.loc[i, 'DEMAND'] / replenishments[i] - 1 for i in range(len(df_demand))]

def max_order_qty_constraint(replenishments):
    return [400 - df_demand.loc[i, 'DEMAND'] / replenishments[i] for i in range(len(df_demand))]

# Initial guess for replenishments per year (start with 1.5 to avoid division by zero)
initial_guess = [1.5] * len(df_demand)

# Combine constraints
constraints = [
    {'type': 'ineq', 'fun': inventory_constraint},
]

# Add order quantity constraints for each SKU
for i in range(len(df_demand)):
    constraints.append({'type': 'ineq', 'fun': lambda r, i=i: min_order_qty_constraint(r)[i]})
    constraints.append({'type': 'ineq', 'fun': lambda r, i=i: max_order_qty_constraint(r)[i]})

# Perform optimization using the 'trust-constr' method
result = minimize(objective, initial_guess, constraints=constraints, method='trust-constr', options={'verbose': 0})

# Display results
if result.success:
    print('Optimal replenishments per year:', result.x)
    print('Minimum total cost:', result.fun)
else:
    print('Optimization failed:', result.message)
    print('Result:', result)


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


KeyboardInterrupt: 